In [21]:
# -------------------------------
# DATA PREPROCESSING
# -------------------------------

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# -------------------------------
#  Load Dataset
# -------------------------------
df = pd.read_csv(r"..\data\raw\marketing_campaign.csv", 
                 encoding='ISO-8859-1', sep="\t")  # Load CSV with proper encoding

# -------------------------------
#  Data Cleaning
# -------------------------------

# Remove duplicate rows
df = df.drop_duplicates()  

# Drop constant columns
df = df.drop(columns=[col for col in df.columns if df[col].nunique() == 1])  

# Impute missing income with median
df['Income'] = df['Income'].fillna(df['Income'].median())  

# Convert to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')  

# Remove unrealistic ages
current_year = pd.Timestamp.today().year
df['Age'] = current_year - df['Year_Birth']
df = df[(df['Age'] >= 18) & (df['Age'] <= 100)]  # Keep only adults <=100

# Remove extreme income outliers using IQR
Q1 = df['Income'].quantile(0.25)
Q3 = df['Income'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
df = df[(df['Income'] >= lower) & (df['Income'] <= upper)]

# Standardize categorical text
df['Education'] = df['Education'].str.strip().str.title()  # Remove extra spaces + unify capitalization
df['Marital_Status'] = df['Marital_Status'].str.strip().str.title()

# Drop unneeded ID/Year_Birth
df = df.drop(columns=['ID','Year_Birth'])

In [22]:
# -------------------------------
#  Feature Engineering
# -------------------------------

df['Customer_Tenure_Days'] = (pd.Timestamp.today() - df['Dt_Customer']).dt.days  # Tenure in days

# Spending columns
spend_cols = ['MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds']

# Log-transform skewed spending
df[spend_cols] = df[spend_cols].apply(lambda x: np.log1p(x))  # Reduce skew

# Total spending across products
df['Total_Spending'] = df[spend_cols].sum(axis=1)

# Total purchases across channels
purchase_cols = ['NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases']
df['Total_Purchases'] = df[purchase_cols].sum(axis=1)

# -------------------------------
#  New Features
# -------------------------------

# Total children at home
df['Children_at_home'] = df['Kidhome'] + df['Teenhome']

# Consolidate Marital_Status into Living_With
df["Living_With"] = df["Marital_Status"].replace({
    "Married":"Partner", "Together":"Partner",
    "Absurd":"Alone", "Widow":"Alone", "Yolo":"Alone",
    "Divorced":"Alone", "Single":"Alone"
})

# Consolidate Education categories
df['Education'] = df['Education'].replace({
    'Basic':'Undergraduate', '2n Cycle':'Undergraduate',
    'Graduation':'Graduate', 'Master':'Postgraduate', 'PhD':'Postgraduate'
})

# Map Living_With to numeric for family size calculation
df['Living_With_Num'] = df['Living_With'].map({"Alone":1,"Partner":2})
df['Living_With_Num'] = df['Living_With_Num'].fillna(1)  # Any unexpected ==> Alone

# Total family members
df['Family_Size'] = df['Living_With_Num'] + df['Children_at_home']

# Income level binning
df['Income_Level'] = pd.cut(df['Income'], bins=[0, 40000, 70000, np.inf],
                            labels=['Low','Medium','High'])  # Group income for categories

# Age group binning
df['Age_Group'] = pd.cut(df['Age'], bins=[17, 30, 45, 60, 100],
                         labels=['Young','Adult','Mid-Age','Senior'])  # Group ages

In [23]:
# -------------------------------
#  DROP REDUNDANT / INTERMEDIATE COLUMNS
# -------------------------------

df = df.drop(columns=[
    'Kidhome',            # Replaced by Children_at_home
    'Teenhome',           # Replaced by Children_at_home
    'Marital_Status',     # Replaced by Living_With
    'Living_With_Num',    # Temporary column for Family_Size
    'Dt_Customer'         # Replaced by Customer_Tenure_Days
])

In [24]:
# -------------------------------
#  Preprocessing for ML
# -------------------------------

num_features = ['Income','Recency','Age','Customer_Tenure_Days',
                'Total_Spending','Total_Purchases','Children_at_home','Family_Size'] + spend_cols + purchase_cols

cat_features = ['Education','Income_Level','Age_Group','Living_With']

# Scale numeric features
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# One-hot encode categorical
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine transformations
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features)
])


In [25]:
# Fit and transform dataset
X_ready = preprocessor.fit_transform(df)

In [26]:
# -------------------------------
#  Save Cleaned & Preprocessed Data
# -------------------------------

# Option 1: Save cleaned DataFrame with all new features (categorical columns still as text)
# Use this if you want a human-readable version for inspection or reporting
df.to_csv(r"..\data\preprocessed\marketing_campaign_cleaned.csv", index=False)

# Option 2: Save ML-ready preprocessed feature matrix (numeric + one-hot encoded)
# Use this for training ML models like K-Means or other algorithms
if hasattr(X_ready, "todense"):  # Convert sparse matrix to dense if needed
    X_ready_dense = X_ready.todense()
else:
    X_ready_dense = X_ready

# Extract proper column names from preprocessor (numeric + one-hot encoded)
feature_names = preprocessor.get_feature_names_out()

# Convert to DataFrame with proper column names
X_ready_df = pd.DataFrame(X_ready_dense, columns=feature_names)

# Save ML-ready matrix
X_ready_df.to_csv(r"..\data\preprocessed\marketing_campaign_preprocessed.csv", index=False)

In [27]:
# -------------------------------
#  Output shapes
# -------------------------------
print("Cleaned dataframe shape:", df.shape)
print("Preprocessed feature matrix shape:", X_ready.shape)
print("Files saved to '..\\data\\preprocessed\\'")

Cleaned dataframe shape: (2229, 30)
Preprocessed feature matrix shape: (2229, 32)
Files saved to '..\data\preprocessed\'


In [29]:
# Display Option 1: cleaned DataFrame with new features
cleaned_df = pd.read_csv(r"..\data\preprocessed\marketing_campaign_cleaned.csv")
print("=== Option 1: Cleaned DataFrame ===")
cleaned_df.head()



=== Option 1: Cleaned DataFrame ===


,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,Response,Age,Customer_Tenure_Days,Total_Spending,Total_Purchases,Children_at_home,Living_With,Family_Size,Income_Level,Age_Group
0,Graduate,58138.0,58,6.455199,4.488636,6.304449,5.153292,4.488636,4.488636,3,...,1,69,4923,31.378848,25,0,Alone,1,Medium,Senior
1,Graduate,46344.0,38,2.484907,0.693147,1.945910,1.098612,0.693147,1.945910,2,...,0,72,4373,8.861634,6,2,Alone,3,Medium,Senior
2,Graduate,71613.0,26,6.056784,3.912023,4.852030,4.718499,3.091042,3.761200,1,...,0,61,4572,26.391579,21,0,Partner,2,High,Senior
3,Graduate,26646.0,26,2.484907,1.609438,3.044522,2.397895,1.386294,1.791759,2,...,0,42,4399,12.714816,8,1,Partner,3,Low,Adult
4,Phd,58293.0,94,5.159055,3.784190,4.779123,3.850148,3.332205,2.772589,5,...,0,45,4421,23.677309,19,1,Partner,3,Medium,Adult


In [ ]:
# Display Option 2: ML-ready preprocessed matrix
preprocessed_df = pd.read_csv(r"..\data\preprocessed\marketing_campaign_preprocessed.csv")
print("\n=== Option 2: Preprocessed ML-ready DataFrame ===")
preprocessed_df.head()  


=== Option 2: Preprocessed ML-ready DataFrame ===


,num__Income,num__Recency,num__Age,num__Customer_Tenure_Days,num__Total_Spending,num__Total_Purchases,num__Children_at_home,num__Family_Size,num__MntWines,num__MntFruits,...,cat__Education_Undergraduate,cat__Income_Level_High,cat__Income_Level_Low,cat__Income_Level_Medium,cat__Age_Group_Adult,cat__Age_Group_Mid-Age,cat__Age_Group_Senior,cat__Age_Group_Young,cat__Living_With_Alone,cat__Living_With_Partner
0,0.316484,0.307314,1.015715,1.529265,1.587551,1.329444,-1.266277,-1.759459,0.985074,1.430723,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.256135,-0.383771,1.272020,-1.190769,-1.292015,-1.163838,1.393978,0.445057,-1.220972,-0.987093,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.970719,-0.798422,0.332234,-0.206611,0.949764,0.804543,-1.266277,-0.657201,0.763699,1.063407,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-1.212506,-0.798422,-1.291031,-1.062186,-0.799259,-0.901387,0.063851,0.445057,-1.220972,-0.403394,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.324010,1.551268,-1.034726,-0.953384,0.602656,0.542092,0.063851,0.445057,0.264887,0.981974,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
